In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PEG residues and terminal PEG residue is with 4 Angstroms of BSA, CONFIGURATION 2

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 0.9 nm PEG restrained Rg 100 ns trajectory C2

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

In [9]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb 
# -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
uC2_n20PEG = mda.Universe("bsapeg_n20waterT2/C2_pegRes/0.9nmpeg_ResC2/bpeg_0.9nmRes.pdb", 
                          "bsapeg_n20waterT2/C2_pegRes/0.9nmpeg_ResC2/C2_0.9nmbpeg.xtc")

In [10]:
uC2_n20PEG

<Universe with 9665 atoms>

Check that we are on the first frame

In [11]:
uC2_n20PEG.trajectory.frame

0

In [12]:
pn20_lenC2 = len(uC2_n20PEG.trajectory)
pn20_lenC2

10001

In [13]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_n20PEGC2 = uC2_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
#list(all_n20PEG)

In [14]:
# Select BSA residues, heavy atoms only 
prot_09nmC2 = uC2_n20PEG.select_atoms("protein and not type H")
prot_09nmC2

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [15]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_lenC2 - 1
s_time = timeit.default_timer()
h2di_09nmC2 = aa_frmcount(prot_09nmC2, all_n20PEGC2, dmax, uC2_n20PEG, start, end)
timeit.default_timer() - s_time

286.29027171991765

In [16]:
len(h2di_09nmC2.keys())

216

In [17]:
pr_res_PEGC2 = list(prot_09nmC2.residues)
ss_res_PEGC2 = [str(row) for row in pr_res_PEGC2]
rkg09_C2 = {key:h2di_09nmC2[key][1] for key, value in h2di_09nmC2.items()}
plgC2_09nm_occ = pd.DataFrame(data=ss_res_PEGC2, columns=["BSA_des_res"])
plgC2_09nm_occ['mda_occ_0.9nmC2'] = plgC2_09nm_occ['BSA_des_res'].map(rkg09_C2)
plgC2_09nm_occ['mda_occ_0.9nmC2'] = plgC2_09nm_occ['mda_occ_0.9nmC2'].replace('nan', np.nan).fillna(0)
plgC2_09nm_occ['mda_occ_0.9nmC2'] = plgC2_09nm_occ['mda_occ_0.9nmC2'].round(2)
plgC2_09nm_occ

,BSA_des_res,mda_occ_0.9nmC2
0,"<Residue ASP, 1>",0.00
1,"<Residue THR, 2>",0.00
2,"<Residue HIS, 3>",0.00
3,"<Residue LYS, 4>",0.00
4,"<Residue SER, 5>",0.00
...,...,...
578,"<Residue GLN, 579>",0.22
579,"<Residue THR, 580>",0.48
580,"<Residue ALA, 581>",0.21
581,"<Residue LEU, 582>",0.06


In [18]:
plgC2_09nm_occ['mda_occ_0.9nmC2'][plgC2_09nm_occ['mda_occ_0.9nmC2'] != 0]

30     0.07
32     0.33
33     0.13
34     0.01
59     0.01
       ... 
578    0.22
579    0.48
580    0.21
581    0.06
582    0.10
Name: mda_occ_0.9nmC2, Length: 148, dtype: float64

In [20]:
bsa_09C2 = np.array(list(prot_09nmC2.resids)) # shape is 4652
mC2_occ_09r = np.array(list(plgC2_09nm_occ['mda_occ_0.9nmC2'])) # shape is 583
mC2_occ = np.zeros(shape=(4653))
atC2_ind = np.where(bsa_09C2[:-1] != bsa_09C2[1:])[0]
atC2_in_nw = np.sort(np.append([0,4653],atC2_ind))
nw_v = 0
for i in range(583):
    b = atC2_in_nw[i+1] +1
    mC2_occ[nw_v:b] = mC2_occ_09r[i]
    nw_v = atC2_in_nw[i+1] + 1 

In [21]:
np.nonzero(mC2_occ)

(array([ 249,  250,  251, ..., 4650, 4651, 4652]),)

### Visualize Occupanct on protein 

In [22]:
prot_09nmC2.occupancies = mC2_occ
prot_09nmC2.occupancies

array([0. , 0. , 0. , ..., 0.1, 0.1, 0.1])

In [23]:
with mda.Writer("prot_09nmC2pegRes.pdb") as pdb:
    pdb.write(prot_09nmC2)

In [24]:
# Frame count and occupancy for each residue
#h2di

In [25]:
len(h2di_09nmC2.keys())

216

### Residue Importance: 0.9 nm restrained C2

In [26]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_09nmPEGC2 = prot_poly_cntmovie(prot_09nmC2, all_n20PEGC2, dmax, uC2_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [27]:
np.save('0.9nmPEG_resC2.npy', trjmap_09nmPEGC2)    # .npy extension is added if not given

In [ ]:
trjmap_09nmPEGC2 = np.load('0.9nmPEG_resC2.npy', allow_pickle=True)

In [28]:
trjmap_09nmPEGC2[0].shape

(583, 60)

In [29]:
np.sum(trjmap_09nmPEGC2[1000][0])

0.0

In [30]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [31]:
ppC2_09nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_09nm_ct[i][j] = np.sum(trjmap_09nmPEGC2[i][j])

In [32]:
ppC2_09nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_09nmtot[i] = np.sum(ppC2_09nm_ct[:,i])
#pp_12nmtot

In [33]:
np.nonzero(ppC2_09nmtot)

(array([ 29,  30,  32,  33,  34,  37,  48,  57,  58,  59,  60,  78,  80,
         81,  82,  83,  84,  85,  86,  88,  91,  92,  95,  96,  99, 103,
        104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 142, 164, 165,
        168, 225, 226, 227, 228, 230, 274, 275, 278, 279, 280, 281, 283,
        298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310,
        311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324,
        327, 328, 330, 331, 334, 335, 336, 339, 350, 354, 358, 359, 360,
        361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373,
        374, 376, 377, 378, 379, 380, 381, 383, 384, 385, 387, 388, 391,
        392, 395, 408, 411, 412, 414, 415, 416, 417, 418, 419, 420, 423,
        440, 457, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 472,
        473, 484, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496,
        497, 498, 499, 500, 501, 502, 503, 506, 507, 508, 509, 518, 521,
        522, 525, 529, 532, 533, 534, 535, 536, 537

In [59]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_09nmtot/np.sum(ppC2_09nmtot), align='center', color='#1D77CF',width=wid, alpha=0.4, label='0.9 nm PEG')
plt.title("BSA in water with PEG Restrained Config 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.16])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block C2

In [35]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [36]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(prot_09nmC2.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [37]:
pr_resC2 = list(prot_09nmC2.residues)
ss_resC2 = [str(row) for row in pr_resC2]
rkg_C2 = {key:h2di_09nmC2[key][0] for key, value in h2di_09nmC2.items()}
plgC2_09nmaa = pd.DataFrame(data=ss_resC2, columns=["BSA_des_res"])
plgC2_09nmaa['mda_plga_frm_0.9nmC2'] = plgC2_09nmaa['BSA_des_res'].map(rkg_C2)
plgC2_09nmaa['BSA_des_res'] = red_bsa
plgC2_09nmaa['mda_plga_frm_0.9nmC2'] = plgC2_09nmaa['mda_plga_frm_0.9nmC2'].replace('nan', np.nan).fillna(0)
plgC2_09nmaa.tail()

,BSA_des_res,mda_plga_frm_0.9nmC2
578,GLN 579,2189.0
579,THR 580,4758.0
580,ALA 581,2137.0
581,LEU 582,572.0
582,ALA 583,992.0


In [38]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

pr_res = list(prot_09nmC2.residues)
ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [39]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_09nmaa.iterrows():
    if r_pl['mda_plga_frm_0.9nmC2'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['pegC2_0.9nm_100ns'] = cpl_l
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nm_100ns
0,LYS,59,59,27
1,ARG,23,23,7
2,HIS,16,16,8
3,ASP,40,40,19
4,GLU,59,59,29
5,SER,23,23,10
6,THR,29,29,17
7,CYS,24,24,7
8,TYR,19,19,7
9,ASN,14,14,4


In [40]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_0.9nm_100ns'].sum()

216

In [41]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [42]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_0.9nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.44353182751540043

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [43]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di_09nmC2.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

Mean Occpancy (1.2 nm Rg): 0.09867777777777777 Occ. std. dev.: 0.18033977285346545


### Calc. fractional contacts for each AA group type 

In [44]:
cd_09nmC2 = frac_cont(h2di_09nmC2)
cd_09nmC2

{'Negative': [42754.0, 48, 192393.0, 0.1612633844708934],
 'Positive': [37966.0, 42, 195253.7142857143, 0.16366122881927675],
 'Polar': [69605.0, 55, 273357.8181818182, 0.22912791490116124],
 'Hydrophobic': [62819.0, 71, 191111.32394366196, 0.16018908645256433],
 'Aromatic': [37880.0, 24, 340920.0, 0.28575838535610426]}

In [ ]:
cd = frac_cont(h2di_09nmC2)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

In [45]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [46]:
fcntC2_rg09nm, prgrp_09nmC2, aa_matx09nmC2 = bavg_frac_cnt(5, prot_09nmC2, all_n20PEGC2, dmax, uC2_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [47]:
fcntC2_rg09nm

{'Negative': array([0.17262195, 0.20270189, 0.12572604, 0.14116249, 0.16113695]),
 'Positive': array([0.19187381, 0.11401607, 0.14686149, 0.20462738, 0.21384093]),
 'Polar': array([0.15661327, 0.2433702 , 0.30364173, 0.18998774, 0.23907269]),
 'Hydrophobic': array([0.16081789, 0.15475734, 0.19718722, 0.14841771, 0.15778089]),
 'Aromatic': array([0.31807308, 0.2851545 , 0.22658352, 0.31580468, 0.22816854]),
 'total_frac': array([0.2238193 , 0.19096509, 0.15400411, 0.2238193 , 0.25051335])}

In [48]:
prgrp_09nmC2

{'Negative': array([array([6.448     , 5.57201005]), array([9.6255    , 3.71500333]),
        array([5.118     , 2.20296981]), array([8.5335    , 3.12424035]),
        array([9.4195    , 3.32738933])], dtype=object),
 'Positive': array([array([13.592     ,  9.30787495]),
        array([20.4045    ,  6.38943501]),
        array([18.7855    ,  3.83438258]),
        array([25.521     ,  5.50622911]), array([26.108    ,  6.7540607])],
       dtype=object),
 'Polar': array([array([7.6565    , 8.68766411]), array([20.902     ,  5.02457919]),
        array([20.7     ,  4.577117]), array([18.0145    ,  4.98490619]),
        array([19.5265    ,  5.24206999])], dtype=object),
 'Hydrophobic': array([array([5.4275    , 4.56078324]), array([9.9215    , 3.77562415]),
        array([13.0525    ,  4.68131859]),
        array([13.96      ,  4.45694963]),
        array([14.066     ,  4.28376517])], dtype=object),
 'Aromatic': array([array([5.413     , 4.25246176]), array([7.1465    , 3.03529863]),
     

In [49]:
fc_09nmC2_mean = np.array([np.mean(fcntC2_rg09nm['Negative']), np.mean(fcntC2_rg09nm['Positive'])
                        ,np.mean(fcntC2_rg09nm['Polar']),np.mean(fcntC2_rg09nm['Hydrophobic'])
                        , np.mean(fcntC2_rg09nm['Aromatic'])])
fc_09nmC2_mean

array([0.16066987, 0.17424394, 0.22653713, 0.16379221, 0.27475686])

In [50]:
fc_09nmC2_std = np.array([np.std(fcntC2_rg09nm['Negative']), np.std(fcntC2_rg09nm['Positive'])
                       ,np.std(fcntC2_rg09nm['Polar']),np.std(fcntC2_rg09nm['Hydrophobic'])
                       , np.std(fcntC2_rg09nm['Aromatic'])])
fc_09nmC2_std

array([0.02650045, 0.03789217, 0.05022114, 0.01719426, 0.04039919])

In [51]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_09nmC2_mean, yerr=fc_09nmC2_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 0.9 nm Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [52]:
# Average of total fraction of contacts
np.mean(fcntC2_rg09nm['total_frac'])

0.2086242299794661

In [53]:
# Std Deviation of total fraction of contacts
np.std(fcntC2_rg09nm['total_frac'])

0.03320145040066979

### Avg no. PEG residues per BSA AA residue group 

In [54]:
prgrp_09nmC2

{'Negative': array([array([6.448     , 5.57201005]), array([9.6255    , 3.71500333]),
        array([5.118     , 2.20296981]), array([8.5335    , 3.12424035]),
        array([9.4195    , 3.32738933])], dtype=object),
 'Positive': array([array([13.592     ,  9.30787495]),
        array([20.4045    ,  6.38943501]),
        array([18.7855    ,  3.83438258]),
        array([25.521     ,  5.50622911]), array([26.108    ,  6.7540607])],
       dtype=object),
 'Polar': array([array([7.6565    , 8.68766411]), array([20.902     ,  5.02457919]),
        array([20.7     ,  4.577117]), array([18.0145    ,  4.98490619]),
        array([19.5265    ,  5.24206999])], dtype=object),
 'Hydrophobic': array([array([5.4275    , 4.56078324]), array([9.9215    , 3.77562415]),
        array([13.0525    ,  4.68131859]),
        array([13.96      ,  4.45694963]),
        array([14.066     ,  4.28376517])], dtype=object),
 'Aromatic': array([array([5.413     , 4.25246176]), array([7.1465    , 3.03529863]),
     

In [55]:
mean_09nmC2 = np.zeros(shape=5)
std_09nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_09nmC2.items():
    mpl_09nmC2 = []
    var_09nmC2 = []
    for i in prgrp_09nmC2[str(key)].flat:
        mpl_09nmC2.append(i[0])
        var_09nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_09nmC2[count] = np.mean(mpl_09nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_09nmC2[count] = np.std(mpl_09nmC2)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [56]:
mean_09nmC2

array([ 7.8289, 20.8822, 17.3599, 11.2855,  7.5147])

In [57]:
std_09nmC2

array([1.7612555 , 4.61733289, 4.95946305, 3.29190156, 1.36879796])

In [58]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_09nmC2, yerr=std_09nmC2, ecolor='black',capsize=5)
plt.title(r'No. of PEG residues 0.9 nm Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PLGA residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PEG contact map 0.9 nm res, Config 2", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.1 nm PEG restrained Rg 100 ns trajectory C2

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [60]:
# Set up the MD Simulation
u11nmC2_n20PEG = mda.Universe("bsapeg_n20waterT2/C2_pegRes/1.1nmpeg_ResC2/bpeg_1.1nmRes.pdb",
                            "bsapeg_n20waterT2/C2_pegRes/1.1nmpeg_ResC2/C2_1.1nmbpeg.xtc")

In [61]:
u11nmC2_n20PEG

<Universe with 9665 atoms>

In [62]:
pn20_11nmC2 = len(u11nmC2_n20PEG.trajectory)
pn20_11nmC2

10001

In [63]:
#Select all the PLGA residues, heavy atoms only 
n20PEG_11nmC2 = u11nmC2_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
n20PEG_11nmC2

<AtomGroup with 183 atoms>

In [64]:
# Select BSA residues, heavy atoms only 
protC2_11nmPEG = u11nmC2_n20PEG.select_atoms("protein and not type H")
protC2_11nmPEG

<AtomGroup with 4653 atoms>

### Contact Analysis

In [65]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_11nmC2 - 1
s_time = timeit.default_timer()
h2di_11nmC2 = aa_frmcount(protC2_11nmPEG, n20PEG_11nmC2, dmax, u11nmC2_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_11nm

196.90344031061977

In [66]:
len(h2di_11nmC2.keys())

171

In [67]:
pr_res_PEGC2 = list(protC2_11nmPEG.residues)
ss_res_PEGC2 = [str(row) for row in pr_res_PEGC2]
rkg_11C2 = {key:h2di_11nmC2[key][1] for key, value in h2di_11nmC2.items()}
plgC2_09nm_occ['mda_occ_1.1nmC2'] = plgC2_09nm_occ['BSA_des_res'].map(rkg_11C2)
plgC2_09nm_occ['mda_occ_1.1nmC2'] = plgC2_09nm_occ['mda_occ_1.1nmC2'].replace('nan', np.nan).fillna(0)
plgC2_09nm_occ['mda_occ_1.1nmC2'] = plgC2_09nm_occ['mda_occ_1.1nmC2'].round(2)
plgC2_09nm_occ

,BSA_des_res,mda_occ_0.9nmC2,mda_occ_1.1nmC2
0,"<Residue ASP, 1>",0.00,0.01
1,"<Residue THR, 2>",0.00,0.00
2,"<Residue HIS, 3>",0.00,0.00
3,"<Residue LYS, 4>",0.00,0.00
4,"<Residue SER, 5>",0.00,0.00
...,...,...,...
578,"<Residue GLN, 579>",0.22,0.02
579,"<Residue THR, 580>",0.48,0.03
580,"<Residue ALA, 581>",0.21,0.03
581,"<Residue LEU, 582>",0.06,0.02


In [68]:
pr_res11nmC2 = list(protC2_11nmPEG.residues)
ss_res11nmC2 = [str(row) for row in pr_res11nmC2]
rkg_11nmC2 = {key:h2di_11nmC2[key][0] for key, value in h2di_11nmC2.items()}
plgC2_1_1nmaa = pd.DataFrame(data=ss_res11nmC2, columns=["BSA_des_res"])
plgC2_1_1nmaa['mda_plga_frm_1.1nmC2'] = plgC2_1_1nmaa['BSA_des_res'].map(rkg_11nmC2)
plgC2_1_1nmaa['BSA_des_res'] = red_bsa
plgC2_1_1nmaa['mda_plga_frm_1.1nmC2'] = plgC2_1_1nmaa['mda_plga_frm_1.1nmC2'].replace('nan', np.nan).fillna(0)
plgC2_1_1nmaa.head()

,BSA_des_res,mda_plga_frm_1.1nmC2
0,ASP 1,86.0
1,THR 2,45.0
2,HIS 3,11.0
3,LYS 4,19.0
4,SER 5,0.0


In [69]:
bsa_11mnC2 = np.array(list(protC2_11nmPEG.resids)) # shape is 4652
m_occ_11nmC2 = np.array(list(plgC2_09nm_occ['mda_occ_1.1nmC2'])) # shape is 583
m_occ_11NewC2 = np.zeros(shape=(4653))
at_ind = np.where(bsa_11mnC2[:-1] != bsa_11mnC2[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_11NewC2[nw_v:b] = m_occ_11nmC2[i]
    nw_v = at_in_nw[i+1] + 1 

In [70]:
np.nonzero(m_occ_11NewC2)

(array([   0,    1,    2,    3,    4,    5,    6,    7,  266,  267,  268,
         269,  270,  271,  272,  273,  274,  465,  466,  467,  468,  469,
         470,  634,  635,  636,  637,  638,  639,  640,  641,  642,  643,
         644,  645,  646,  647,  648,  649,  650,  651,  652,  653,  654,
         655,  656,  657,  658,  659,  660,  673,  674,  675,  676,  885,
         886,  887,  888,  889,  890,  891,  892,  893,  894,  895,  896,
         897,  898,  899, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153,
        1154, 1155, 1156, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399,
        2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410,
        2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421,
        2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431, 2432,
        2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443,
        2444, 2445, 2446, 2447, 2448, 2449, 2450, 2470, 2471, 2472, 2473,
        2474, 2475, 2476, 2477, 2478, 

### Visualize Occupanct on protein 

In [71]:
protC2_11nmPEG.occupancies = m_occ_11NewC2
protC2_11nmPEG.occupancies[0:33]

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [72]:
with mda.Writer("prot_11nmC2pegRes.pdb") as pdb:
    pdb.write(protC2_11nmPEG)

### Residue Importance: 1.1 nm restrained 

In [73]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_11nmPEGC2 = prot_poly_cntmovie(protC2_11nmPEG, n20PEG_11nmC2, dmax, u11nmC2_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [75]:
#trjmap_11nmPEGC2 = np.load('1.1nm_PEGres.npy', allow_pickle=True)

In [76]:
np.save('1.1nmPEG_resC2.npy', trjmap_11nmPEGC2)    # .npy extension is added if not given

In [77]:
trjmap_11nmPEGC2[0].shape

(583, 60)

In [78]:
np.sum(trjmap_11nmPEGC2[1000][0])

0.0

In [79]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [80]:
ppC2_11nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_11nm_ct[i][j] = np.sum(trjmap_11nmPEGC2[i][j])

In [81]:
ppC2_11nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_11nmtot[i] = np.sum(ppC2_11nm_ct[:,i])
#pp_12nmtot

In [82]:
np.nonzero(ppC2_11nmtot)

(array([  0,   1,   2,   3,   8,  32,  33,  34,  36,  37,  40,  55,  57,
         58,  59,  62,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  87,  88, 110, 111, 112, 114, 138, 139, 142, 227, 231,
        257, 261, 264, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305,
        308, 309, 310, 311, 312, 315, 318, 319, 331, 332, 335, 336, 337,
        338, 339, 343, 344, 347, 357, 358, 359, 360, 361, 362, 363, 364,
        365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
        379, 380, 381, 382, 383, 384, 385, 387, 388, 390, 391, 392, 393,
        394, 395, 396, 397, 398, 399, 400, 401, 403, 404, 405, 407, 408,
        411, 412, 415, 433, 436, 437, 438, 439, 440, 441, 442, 443, 444,
        445, 462, 463, 464, 465, 466, 467, 468, 469, 472, 476, 487, 488,
        489, 490, 491, 494, 516, 520, 523, 536, 537, 538, 539, 540, 541,
        543, 546, 547, 550, 551, 554, 555, 572, 575, 576, 578, 579, 580,
        581, 582]),)

In [87]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_11nmtot/np.sum(ppC2_11nmtot), align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtot/np.sum(pp_09nmtot), align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG Restrained Config 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block C2

In [88]:
apl_11nmC2 = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_1_1nmaa.iterrows():
    if r_pl['mda_plga_frm_1.1nmC2'] != 0:
        apl_11nmC2.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_11nmC2 = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_11nmC2)):
        if r_a['Amino_acids'] in apl_11nmC2[i]:
            count += 1
    cpl_11nmC2.append(count)      
        
aa_count['pegC2_1.1nm_100ns'] = cpl_11nmC2
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nm_100ns,pegC2_1.1nm_100ns
0,LYS,59,59,27,19
1,ARG,23,23,7,6
2,HIS,16,16,8,7
3,ASP,40,40,19,17
4,GLU,59,59,29,20
5,SER,23,23,10,7
6,THR,29,29,17,13
7,CYS,24,24,7,7
8,TYR,19,19,7,7
9,ASN,14,14,4,3


In [89]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_1.1nm_100ns'].sum()

171

In [90]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [91]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_1.1nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.351129363449692

In [92]:
# Mean occupancy and std deviation 
ll_mo11 = [value[1] for key, value in h2di_11nmC2.items()]
print("Mean Occpancy (1.1 nm Rg): "+str(np.mean(ll_mo11)), "Occ. std. dev.: "+str(np.std(ll_mo11)))

Mean Occpancy (1.1 nm Rg): 0.06964912280701753 Occ. std. dev.: 0.15552870229019672


In [93]:
cd_11nmC2 = frac_cont(h2di_11nmC2)
cd_11nmC2

{'Negative': [20381.0, 37, 94193.27027027028, 0.11398353248782438],
 'Positive': [32352.0, 32, 172881.0, 0.20920376820431016],
 'Polar': [35791.0, 46, 133049.15217391305, 0.16100314083775139],
 'Hydrophobic': [30576.0, 56, 93366.0, 0.11298245048422687],
 'Aromatic': [33094.0, 17, 332886.70588235295, 0.40282710798588706]}

### Calc. fractional contacts for each AA group type C2

In [94]:
fcntC2_rg11nm, prgrp_11nmC2, aa_matx_11nmC2 = bavg_frac_cnt(5, protC2_11nmPEG, n20PEG_11nmC2, dmax 
                                                            ,u11nmC2_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [95]:
fcntC2_rg11nm

{'Negative': array([0.11159174, 0.10728814, 0.13777792, 0.13197951, 0.10393028]),
 'Positive': array([0.2252543 , 0.26023861, 0.31325555, 0.16238616, 0.14681183]),
 'Polar': array([0.20436304, 0.18348181, 0.18167001, 0.23755254, 0.19260404]),
 'Hydrophobic': array([0.12205346, 0.10793171, 0.09649502, 0.14282247, 0.15118941]),
 'Aromatic': array([0.33673746, 0.34105973, 0.27080151, 0.32525933, 0.40546443]),
 'total_frac': array([0.11498973, 0.13552361, 0.07392197, 0.19096509, 0.17043121])}

In [96]:
fc_11nmC2_mean = np.array([np.mean(fcntC2_rg11nm['Negative']), np.mean(fcntC2_rg11nm['Positive'])
                        ,np.mean(fcntC2_rg11nm['Polar']),np.mean(fcntC2_rg11nm['Hydrophobic'])
                        , np.mean(fcntC2_rg11nm['Aromatic'])])
fc_11nmC2_mean

array([0.11851352, 0.22158929, 0.19993429, 0.12409841, 0.33586449])

In [97]:
fc_11nmC2_std = np.array([np.std(fcntC2_rg11nm['Negative']), np.std(fcntC2_rg11nm['Positive'])
                       ,np.std(fcntC2_rg11nm['Polar']),np.std(fcntC2_rg11nm['Hydrophobic'])
                       , np.std(fcntC2_rg11nm['Aromatic'])])
fc_11nmC2_std

array([0.01370431, 0.06165482, 0.02045848, 0.02055239, 0.04293615])

In [107]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_09nmC2_mean, width, yerr=fc_09nmC2_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_11nmC2_mean, width, yerr=fc_11nmC2_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [99]:
np.mean(fcntC2_rg11nm['total_frac'])

0.13716632443531826

In [100]:
np.std(fcntC2_rg11nm['total_frac'])

0.04121943245215476

### Avg no. PEG residues per BSA AA residue group 

In [101]:
prgrp_11nmC2

{'Negative': array([array([2.6115    , 2.89664767]), array([4.0725    , 2.70023031]),
        array([4.8155   , 2.4017618]), array([4.336     , 2.70963171]),
        array([3.463     , 2.63279908])], dtype=object),
 'Positive': array([array([6.1665    , 4.60660154]), array([14.8935   ,  5.0384678]),
        array([16.982     ,  4.67190282]),
        array([14.6535    ,  6.68636207]), array([9.932     , 5.38194909])],
       dtype=object),
 'Polar': array([array([5.247     , 4.06607809]), array([5.5215    , 2.73871097]),
        array([4.409     , 1.89781954]), array([12.387     ,  5.44602892]),
        array([11.8195    ,  4.03533391])], dtype=object),
 'Hydrophobic': array([array([3.075     , 2.63104067]), array([3.881     , 2.45353602]),
        array([3.433     , 2.12096935]), array([7.6705    , 4.59847037]),
        array([9.3245   , 3.5164186])], dtype=object),
 'Aromatic': array([array([3.9355    , 3.01186649]), array([7.7465  , 3.080136]),
        array([7.3125   , 2.7276077]), 

In [102]:
mean_11nmC2 = np.zeros(shape=5)
std_11nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_11nmC2.items():
    mpl_11nmC2 = []
    var_11nmC2 = []
    for i in prgrp_11nmC2[str(key)].flat:
        mpl_11nmC2.append(i[0])
        var_11nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_11nmC2[count] = np.mean(mpl_11nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_11nmC2[count] = np.std(mpl_11nmC2)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [103]:
mean_11nmC2

array([ 3.8597, 12.5255,  7.8768,  5.4768,  7.8281])

In [104]:
std_11nmC2

array([0.76171823, 3.92834121, 3.47493051, 2.534145  , 2.49797122])

In [106]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_09nmC2, width, yerr=std_09nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_11nmC2, width, yerr=std_11nmC2, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PEG residues, Rg restrained, Config. 2', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'No. of PEG residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PEG residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.4 nm restrained Rg PEG 100 ns trajectory C2

### Contact Analyis

In [108]:
# Set up the MD Simulation
u14nmC2_n20PEG = mda.Universe("bsapeg_n20waterT2/C2_pegRes/1.4nmpeg_ResC2/bpeg_1.4nmRes.pdb"
                              , "bsapeg_n20waterT2/C2_pegRes/1.4nmpeg_ResC2/C2Res_1.4nmbpeg.xtc")

In [109]:
u14nmC2_n20PEG

<Universe with 9665 atoms>

In [110]:
pn20_len14nmC2 = len(u14nmC2_n20PEG.trajectory)
pn20_len14nmC2

10001

In [111]:
#Select all the PLGA residues, heavy atoms only 
pn20_14nmC2 = u14nmC2_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
pn20_14nmC2

<AtomGroup with 183 atoms>

In [113]:
# Select BSA residues, heavy atoms only 
prot_14nmC2 = u14nmC2_n20PEG.select_atoms("protein and not type H")
prot_14nmC2

<AtomGroup with 4653 atoms>

In [114]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len14nmC2 - 1
s_time = timeit.default_timer()
h2di_14nmC2 = aa_frmcount(prot_14nmC2, pn20_14nmC2, dmax, u14nmC2_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_14nm

180.1104526836425

In [156]:
#h2di_14nmC2

In [116]:
len(h2di_14nmC2.keys())

244

In [117]:
pr_res14nmC2 = list(prot_14nmC2.residues)
ss_res14nmC2 = [str(row) for row in pr_res14nmC2]
rkg_14nmC2 = {key:h2di_14nmC2[key][0] for key, value in h2di_14nmC2.items()}
plgC2_14nmaa = pd.DataFrame(data=ss_res14nmC2, columns=["BSA_des_res"])
plgC2_14nmaa['mda_plga_frm_1.4nmC2'] = plgC2_14nmaa['BSA_des_res'].map(rkg_14nmC2)
plgC2_14nmaa['BSA_des_res'] = red_bsa
plgC2_14nmaa['mda_plga_frm_1.4nmC2'] = plgC2_14nmaa['mda_plga_frm_1.4nmC2'].replace('nan', np.nan).fillna(0)
plgC2_14nmaa.head()

,BSA_des_res,mda_plga_frm_1.4nmC2
0,ASP 1,923.0
1,THR 2,1643.0
2,HIS 3,1149.0
3,LYS 4,1452.0
4,SER 5,5.0


In [119]:
# Extract mean occupancy values
pr_res_14ur = list(prot_14nmC2.residues)
ss_res_14ur = [str(row) for row in pr_res_14ur]
rkg_14ur = {key:h2di_14nmC2[key][1] for key, value in h2di_14nmC2.items()}
plgC2_09nm_occ['mda_occ_1.4nmC2'] = plgC2_09nm_occ['BSA_des_res'].map(rkg_14ur)
plgC2_09nm_occ['mda_occ_1.4nmC2'] = plgC2_09nm_occ['mda_occ_1.4nmC2'].replace('nan', np.nan).fillna(0)
plgC2_09nm_occ['mda_occ_1.4nmC2'] = plgC2_09nm_occ['mda_occ_1.4nmC2'].round(2)
plgC2_09nm_occ

,BSA_des_res,mda_occ_0.9nmC2,mda_occ_1.1nmC2,mda_occ_1.4nmC2
0,"<Residue ASP, 1>",0.00,0.01,0.09
1,"<Residue THR, 2>",0.00,0.00,0.16
2,"<Residue HIS, 3>",0.00,0.00,0.11
3,"<Residue LYS, 4>",0.00,0.00,0.15
4,"<Residue SER, 5>",0.00,0.00,0.00
...,...,...,...,...
578,"<Residue GLN, 579>",0.22,0.02,0.00
579,"<Residue THR, 580>",0.48,0.03,0.04
580,"<Residue ALA, 581>",0.21,0.03,0.03
581,"<Residue LEU, 582>",0.06,0.02,0.00


In [120]:
bsa_14nmC2 = np.array(list(prot_14nmC2.resids)) # shape is 4652
m_occ_14nmC2 = np.array(list(plgC2_09nm_occ['mda_occ_1.4nmC2'])) # shape is 583
m_occ_14NewC2 = np.zeros(shape=(4653))
atC2_ind = np.where(bsa_14nmC2[:-1] != bsa_14nmC2[1:])[0]
atC2_in_nw = np.sort(np.append([0,4653],atC2_ind))
nw_v = 0
for i in range(583):
    b = atC2_in_nw[i+1] +1
    m_occ_14NewC2[nw_v:b] = m_occ_14nmC2[i]
    nw_v = atC2_in_nw[i+1] + 1 

In [121]:
np.nonzero(m_occ_14NewC2)

(array([   0,    1,    2, ..., 4636, 4637, 4638]),)

### Visualize Occupanct on protein C2

In [122]:
prot_14nmC2.occupancies = m_occ_14NewC2
prot_14nmC2.occupancies

array([0.09, 0.09, 0.09, ..., 0.  , 0.  , 0.  ])

In [123]:
with mda.Writer("prot_14nmC2pegRes.pdb") as pdb:
    pdb.write(prot_14nmC2)

### Residue Importance: 1.4 nm restrained 

In [124]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_14nmPEGC2 = prot_poly_cntmovie(prot_14nmC2, pn20_14nmC2, dmax, u14nmC2_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [125]:
np.save('1.4nmPEG_resC2.npy', trjmap_14nmPEGC2)    # .npy extension is added if not given

In [ ]:
trjmap_14nmPEGC2 = np.load('1.4nmPEG_resC2.npy', allow_pickle=True)
trjmap_14nmPEGC2

In [126]:
trjmap_14nmPEGC2[0].shape

(583, 60)

In [127]:
np.sum(trjmap_14nmPEGC2[1000][0])

0.0

In [128]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [129]:
ppC2_14nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_14nm_ct[i][j] = np.sum(trjmap_14nmPEGC2[i][j])

In [130]:
ppC2_14nmtot = np.zeros(shape=(583))
for i in range(583):
    ppC2_14nmtot[i] = np.sum(ppC2_14nm_ct[:,i])
#pp_12nmtot

In [131]:
np.nonzero(ppC2_14nmtot)

(array([  0,   1,   2,   3,   4,   7,   8,  10,  11,  12,  13,  14,  15,
         16,  17,  32,  40,  41,  44,  45,  47,  48,  50,  52,  53,  54,
         55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  66,  67,  68,
         71,  72,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
         85,  87,  88,  90,  91,  92,  93,  94,  95,  96,  97,  99, 100,
        103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 126, 127,
        128, 129, 130, 157, 160, 161, 164, 165, 168, 202, 203, 204, 205,
        206, 207, 210, 226, 227, 228, 230, 231, 234, 235, 238, 254, 257,
        260, 261, 263, 264, 265, 266, 267, 268, 274, 275, 277, 278, 279,
        280, 281, 282, 284, 298, 299, 300, 301, 302, 304, 305, 308, 309,
        311, 312, 315, 322, 323, 353, 356, 357, 360, 361, 362, 363, 365,
        366, 369, 370, 373, 376, 377, 381, 384, 385, 388, 389, 391, 392,
        395, 397, 398, 399, 400, 405, 408, 409, 411, 412, 415, 417, 419,
        428, 430, 462, 463, 464, 465, 466, 467, 468

In [132]:
a_peg = np.sum(ppC2_14nmtot)

In [133]:
b_peg = np.sum(ppC2_11nmtot)

In [134]:
c_peg = np.sum(ppC2_09nmtot)

In [ ]:
plt.close('all')

In [158]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
#plt.bar(y_pos, ppC2_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
#plt.bar(y_pos+0.25, ppC2_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos+0.3, ppC2_14nmtot/a_peg, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='1.4 nm PEG')
#plt.bar(y_pos+0.25, pp_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG restrained, Config 2, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block C2

In [137]:
apl_14nmC2 = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_14nmaa.iterrows():
    if r_pl['mda_plga_frm_1.4nmC2'] != 0:
        apl_14nmC2.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_14nmC2 = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_14nmC2)):
        if r_a['Amino_acids'] in apl_14nmC2[i]:
            count += 1
    cpl_14nmC2.append(count)      
        
aa_count['pegC2_1.4nm_100ns'] = cpl_14nmC2
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nm_100ns,pegC2_1.1nm_100ns,pegC2_1.4nm_100ns
0,LYS,59,59,27,19,36
1,ARG,23,23,7,6,5
2,HIS,16,16,8,7,10
3,ASP,40,40,19,17,27
4,GLU,59,59,29,20,34
5,SER,23,23,10,7,11
6,THR,29,29,17,13,17
7,CYS,24,24,7,7,8
8,TYR,19,19,7,7,4
9,ASN,14,14,4,3,7


In [138]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_1.4nm_100ns'].sum()

244

In [139]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [140]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_1.4nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.5010266940451745

In [141]:
# Mean occupancy and std deviation 
ll_mo14nm = [value[1] for key, value in h2di_14nmC2.items()]
print("Mean Occpancy (1.4 nm Rg): "+str(np.mean(ll_mo14nm)), "Occ. std. dev.: "+str(np.std(ll_mo14nm)))

Mean Occpancy (1.4 nm Rg): 0.02847704918032787 Occ. std. dev.: 0.049686123482341624


### Calc. fractional contacts for each AA group type 

In [142]:
fcntC2_rg14nm, prgrp_14nmC2, aa_matx_14nmC2 = bavg_frac_cnt(5, prot_14nmC2, pn20_14nmC2, dmax, u14nmC2_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [143]:
fcntC2_rg14nm

{'Negative': array([0.1864835 , 0.37109474, 0.18534373, 0.12540943, 0.14485804]),
 'Positive': array([0.17727823, 0.23205062, 0.18680328, 0.29991888, 0.11968638]),
 'Polar': array([0.19829477, 0.21749752, 0.20174167, 0.26690394, 0.12497352]),
 'Hydrophobic': array([0.20997254, 0.09683423, 0.31451423, 0.08859468, 0.20576785]),
 'Aromatic': array([0.22797096, 0.08252289, 0.11159708, 0.21917307, 0.40471421]),
 'total_frac': array([0.20944559, 0.18480493, 0.15605749, 0.11704312, 0.18685832])}

In [144]:
fc_14nmC2_mean = np.array([np.mean(fcntC2_rg14nm['Negative']), np.mean(fcntC2_rg14nm['Positive'])
                        ,np.mean(fcntC2_rg14nm['Polar']),np.mean(fcntC2_rg14nm['Hydrophobic'])
                        , np.mean(fcntC2_rg14nm['Aromatic'])])
fc_14nmC2_mean

array([0.20263789, 0.20314748, 0.20188228, 0.18313671, 0.20919564])

In [145]:
fc_14nmC2_std = np.array([np.std(fcntC2_rg14nm['Negative']), np.std(fcntC2_rg14nm['Positive'])
                       ,np.std(fcntC2_rg14nm['Polar']),np.std(fcntC2_rg14nm['Hydrophobic'])
                       , np.std(fcntC2_rg14nm['Aromatic'])])
fc_14nmC2_std

array([0.08745343, 0.06017009, 0.04560592, 0.08352089, 0.11335875])

In [157]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '700px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc_09nmC2_mean, width, yerr=fc_09nmC2_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_11nmC2_mean, width, yerr=fc_11nmC2_std, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc_14nmC2_mean, width, yerr=fc_14nmC2_std, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [147]:
np.mean(fcntC2_rg14nm['total_frac'])

0.17084188911704312

In [148]:
np.std(fcntC2_rg14nm['total_frac'])

0.03179504205477768

### Avg no. PEG residues per BSA AA residue group 

In [149]:
prgrp_14nmC2

{'Negative': array([array([4.259     , 3.64100522]), array([5.781    , 3.0016727]),
        array([3.351     , 2.37882303]), array([1.0845    , 2.05313413]),
        array([3.876     , 2.81542608])], dtype=object),
 'Positive': array([array([2.2835    , 2.15130838]), array([6.137     , 2.91139674]),
        array([5.755     , 2.94176393]), array([2.8785    , 4.65711689]),
        array([4.1905    , 2.81695043])], dtype=object),
 'Polar': array([array([3.574     , 4.38560418]), array([4.295     , 2.90963486]),
        array([3.781     , 3.09807021]), array([1.881     , 2.42689905]),
        array([2.5285    , 2.36160703])], dtype=object),
 'Hydrophobic': array([array([3.743     , 3.61523872]), array([2.3195    , 3.21191839]),
        array([4.8585    , 5.30975308]), array([0.614     , 1.53101404]),
        array([7.9435    , 4.02868561])], dtype=object),
 'Aromatic': array([array([1.212     , 1.51031652]), array([0.495     , 1.19288516]),
        array([0.73      , 1.04599235]), array([

In [150]:
mean_14nmC2 = np.zeros(shape=5)
std_14nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrp_14nmC2.items():
    mpl_14nmC2 = []
    var_14nmC2 = []
    for i in prgrp_14nmC2[str(key)].flat:
        mpl_14nmC2.append(i[0])
        var_14nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_14nmC2[count] = np.mean(mpl_14nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_14nmC2[count] = np.std(mpl_14nmC2)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [151]:
mean_14nmC2

array([3.6703, 4.2489, 3.2119, 3.8957, 1.3973])

In [152]:
std_14nmC2

array([1.52527039, 1.52167426, 0.8793665 , 2.47341656, 1.10582538])

In [153]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_09nmC2, width, yerr=std_09nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_11nmC2, width, yerr=std_11nmC2, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_14nmC2, width, yerr=std_14nmC2, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PEG residues, Rg restrained, Config 2', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False)
plt.ylabel(r'No. of PEG residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PEG residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PEG contact map 1.4 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()